## 序贯（Sequential）模型
序贯模型是多个网络层的线性堆叠。

Dense层是常用的全连接层，所实现的运算是`output = activation(dot(input, kernel)+bias)`。其中`activatio`n是逐元素计算的激活函数，`kernel`是本层的权值矩阵，`bias`为偏置向量，只有当`use_bias=True`才会添加。

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential([
    Dense(input_shape=(32,), units=784),
    Activation('relu'),
    Dense(units=10),
    Activation('softmax')
])

也可以通过`.add()`方法一个个地将layer加入模型中：

In [7]:
model = Sequential()
model.add(Dense(units=32, input_shape=(784, )))
model.add(Activation('relu'))

### 指定输入数据的shape
Sequential的第一层需要接收一个关于输入模型shape的参数，后面各个层可以自动推导出中间数据的shape。有几种方法指定第一层输入数据的shape。

+ 传递一个`input_shape`关键字给第一层，`input_shape`是一个tuple型数据，其中可以填入**None**，表示此位置**可能是任何正整数**，数据的batch_size不应包含在其中。
+ 有些2D层，如Dense，支持通过指定其输入维度`input_dim`来隐含的指定输入数据shape。一些3D的时域层支持通过参数`input_dim`和`input_length`来指定输入shape。
+ 如果你需要为输入指定一个固定大小的`batch_size`（常用于stateful RNN网络），可以传递batch_size参数到一个层中，例如你想指定输入张量的batch大小是32，数据shape是（6，8），则你需要传递`batch_size=32`和`input_shape=(6,8)`。

### 编译
`compile`接收三个参数。
+ 优化器optimizer：可以传入预定义的优化器名，如`rmsprop`，`adagrad`。或一个`Optimizer`类的对象。
+ 损失函数loss
+ 指标列表metrics。对于分类问题，一般将该列表设置为`metrics=['accuracy']`

In [11]:
# For a multi-class classification problem
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# For a binary classification problem
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['accuracy'])

# For a mean squared error regression problem
model.compile(optimizer='rmsprop',
             loss='mse')

# 自定义指标
import keras.backend as K

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['accuracy', mean_pred])

### 训练
Keras以Numpy数组作为输入数据和标签的数据类型。训练模型一般使用fit函数，该函数的详情见这里。下面是一些例子。

In [15]:
# For a single-input model with 2 classes (binary classification):
model = Sequential()
model.add(Dense(units=32, activation='relu', input_dim=100))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Generate dummy data
import numpy as np
data = np.random.random((1000, 100))
labels = np.random.randint(2, size=(1000, 1))

# Train the model, iterating on the data in batches of 32 samples
model.fit(data, labels, epochs=10, batch_size=32)

Epoch 1/10
1000/1000 [==============================] - 1s 1ms/step - loss: 0.7033 - acc: 0.4960
Epoch 2/10
1000/1000 [==============================] - 0s 33us/step - loss: 0.6946 - acc: 0.5190
Epoch 3/10
1000/1000 [==============================] - 0s 34us/step - loss: 0.6881 - acc: 0.5390
Epoch 4/10
1000/1000 [==============================] - 0s 34us/step - loss: 0.6872 - acc: 0.5320
Epoch 5/10
1000/1000 [==============================] - 0s 35us/step - loss: 0.6828 - acc: 0.5570
Epoch 6/10
1000/1000 [==============================] - 0s 34us/step - loss: 0.6785 - acc: 0.5670
Epoch 7/10
1000/1000 [==============================] - 0s 32us/step - loss: 0.6738 - acc: 0.5800
Epoch 8/10
1000/1000 [==============================] - 0s 34us/step - loss: 0.6753 - acc: 0.5710
Epoch 9/10
1000/1000 [==============================] - 0s 32us/step - loss: 0.6693 - acc: 0.5810
Epoch 10/10
1000/1000 [==============================] - 0s 34us/step - loss: 0.6680 - acc: 0.6010


In [20]:
# For a single-input model with 10 classes (categorical classification):

model = Sequential()
model.add(Dense(units=32, activation='relu', input_dim=100))
model.add(Dense(units=10, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

import keras
import numpy as np
data = np.random.random((1000, 100))
labels = np.random.randint(10, size=(1000, 1))

# Convert labels to categorical onehot encoding
one_hot_labels = keras.utils.to_categorical(labels, num_classes=10)

model.fit(data, one_hot_labels, epochs=20, batch_size=32)

Epoch 1/50
1000/1000 [==============================] - 0s 421us/step - loss: 2.3552 - acc: 0.0870
Epoch 2/50
1000/1000 [==============================] - 0s 37us/step - loss: 2.3123 - acc: 0.1080
Epoch 3/50
1000/1000 [==============================] - 0s 32us/step - loss: 2.3011 - acc: 0.1000
Epoch 4/50
1000/1000 [==============================] - 0s 36us/step - loss: 2.2903 - acc: 0.1190
Epoch 5/50
1000/1000 [==============================] - 0s 34us/step - loss: 2.2834 - acc: 0.1350
Epoch 6/50
1000/1000 [==============================] - 0s 35us/step - loss: 2.2748 - acc: 0.1350
Epoch 7/50
1000/1000 [==============================] - 0s 37us/step - loss: 2.2640 - acc: 0.1410
Epoch 8/50
1000/1000 [==============================] - 0s 33us/step - loss: 2.2582 - acc: 0.1550
Epoch 9/50
1000/1000 [==============================] - 0s 39us/step - loss: 2.2480 - acc: 0.1610
Epoch 10/50
1000/1000 [==============================] - 0s 35us/step - loss: 2.2390 - acc: 0.1680
Epoch 11/50
1000/1

## 案例
### 基于多层感知器的softmax多分类

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

import numpy as np
import keras.backend as K

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

x_train = np.random.random((1000, 20))
y_train = keras.utils.to_categorical(np.random.randint(10, size=(1000, 1)), num_classes=10)
x_test = np.random.random((100, 20))
y_test = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)

model = Sequential()

model.add(Dense(units=64, activation='relu', input_dim=20))
model.add(Dropout(0.5))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy', mean_pred])

model.fit(x_train, y_train, epochs=10, batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)
print(score)

Epoch 1/10
1000/1000 [==============================] - 0s 399us/step - loss: 2.3576 - acc: 0.0920 - mean_pred: 0.1000
Epoch 2/10
1000/1000 [==============================] - 0s 22us/step - loss: 2.3372 - acc: 0.0980 - mean_pred: 0.1000
Epoch 3/10
1000/1000 [==============================] - 0s 23us/step - loss: 2.3095 - acc: 0.1150 - mean_pred: 0.1000
Epoch 4/10
1000/1000 [==============================] - 0s 25us/step - loss: 2.3139 - acc: 0.0960 - mean_pred: 0.1000
Epoch 5/10
1000/1000 [==============================] - 0s 23us/step - loss: 2.3128 - acc: 0.1050 - mean_pred: 0.1000
Epoch 6/10
1000/1000 [==============================] - 0s 25us/step - loss: 2.3201 - acc: 0.0860 - mean_pred: 0.1000
Epoch 7/10
1000/1000 [==============================] - 0s 22us/step - loss: 2.3006 - acc: 0.1200 - mean_pred: 0.1000
Epoch 8/10
1000/1000 [==============================] - 0s 27us/step - loss: 2.3148 - acc: 0.1100 - mean_pred: 0.1000
Epoch 9/10
1000/1000 [==============================] -

### 类似VGG的卷积神经网络

1. Conv2D
Conv2D中padding参数为valid或same。默认为valid。

2. Dropout
Dropout的比例是断开的神经元比例，也就是去除0.25（剩下0.75）。

3. Flatten
Flatten层将输入压平，把多维的输入一维化。

4. model.evaluate()
返回的结果对应compile中metrics参数列表，从第二个值开始对应，第一个值为loss值。假设将返回值变量赋给score，则`score[0] = loss, score[1] = metrics[0]，score[2] = metrics[1]`，以此类推。

In [2]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import SGD

In [3]:
x_train = np.random.random((100, 100, 100, 3))
y_train = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)
x_test = np.random.random((20, 100, 100, 3))
y_test = keras.utils.to_categorical(np.random.randint(10, size=(20, 1)), num_classes=10)

In [7]:
model = Sequential()
# 这里使用了32个size为3*3卷积层
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# padding参数为valid或same。默认为valid
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# dropout的比例是断开的神经元比例，也就是去除0.25（剩下0.75）
model.add(Dropout(0.25))

# Flatten层将输入压平，把多维的输入一维化
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=10)
score = model.evaluate(x_test, y_test, batch_size=32)

Epoch 1/10
100/100 [==============================] - 6s 58ms/step - loss: 2.3940 - acc: 0.1300
Epoch 2/10
100/100 [==============================] - 5s 52ms/step - loss: 2.3822 - acc: 0.1300
Epoch 3/10
100/100 [==============================] - 5s 52ms/step - loss: 2.2893 - acc: 0.1600
Epoch 4/10
100/100 [==============================] - 6s 59ms/step - loss: 2.3117 - acc: 0.1600
Epoch 5/10
100/100 [==============================] - 5s 54ms/step - loss: 2.2886 - acc: 0.1200
Epoch 6/10
100/100 [==============================] - 5s 53ms/step - loss: 2.2717 - acc: 0.1800
Epoch 7/10
100/100 [==============================] - 5s 51ms/step - loss: 2.3134 - acc: 0.1900
Epoch 8/10
100/100 [==============================] - 6s 55ms/step - loss: 2.2906 - acc: 0.1300
Epoch 9/10
100/100 [==============================] - 5s 54ms/step - loss: 2.2776 - acc: 0.1600
Epoch 10/10
20/20 [==============================] - 0s 16ms/step
